In [36]:
from chembl_webresource_client.new_client import new_client
import pandas as pd


molecule = new_client.molecule


chembl_ids = ['CHEMBL485', 'CHEMBL70', 'CHEMBL113', 'CHEMBL3', 'CHEMBL517712', 'CHEMBL170', 'CHEMBL4596781', 'CHEMBL4803817', 'CHEMBL4597035', 'CHEMBL5028438', 'CHEMBL5028232']


def get_molecules(chembl_ids):
    molecule_dict={}
    for chembl_id in chembl_ids:
        try: 
            response = molecule.get(chembl_id)
            molecule_dict[chembl_id]=response  
        except Exception as e:
            return str(e) 
        
    return molecule_dict


def check_natural(molecule):
    return molecule['natural_product']
    
def check_analgesic(molecule):
    return 1 if molecule['indication_class'] and 'Analgesic' in molecule['indication_class'] else 0
    
molecule_props=get_molecules(chembl_ids)    


results = [(key, check_natural(value), check_analgesic(value)) for key,value in molecule_props.items()]

results_df = pd.DataFrame(results, columns=['CHEMBL_ID', 'Is_Natural_Product', 'Is_Anaglesic'])
results_df.to_csv('./data/natural_products3.csv', index=False)